In [ ]:
from google.colab import files
import os

In [ ]:
# Buat folder untuk menyimpan data PDF
if not os.path.exists('data'):
    os.makedirs('data')

print("Silakan upload file PDF.")
print("Setelah di-upload, file akan disimpan di folder 'data'.")

# Upload file
uploaded = files.upload()

# Pindahkan file yang di-upload ke folder data
for filename in uploaded.keys():
    os.rename(filename, os.path.join('data', filename))

print("\nFile berhasil di-upload dan dipindahkan ke folder 'data'!")

Silakan upload file PDF.
Setelah di-upload, file akan disimpan di folder 'data'.


Saving Attention Is All You Need.pdf to Attention Is All You Need.pdf

File berhasil di-upload dan dipindahkan ke folder 'data'!


In [ ]:
!pip install -q langchain langchain-community langchain-groq sentence-transformers faiss-cpu pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyMuPDFLoader

In [ ]:
# Tentukan path ke paper
pdf_path = "/content/data/Attention Is All You Need.pdf" # Ganti nama file jika berbeda

In [ ]:
# 1. Load PDF menggunakan PyMuPDFLoader dari LangChain
print(f"🔄 Membaca file PDF dari: {pdf_path}")
loader = PyMuPDFLoader(pdf_path)
documents = loader.load()
print(f"✅ PDF berhasil dibaca. Ditemukan {len(documents)} halaman.")

🔄 Membaca file PDF dari: /content/data/Attention Is All You Need.pdf
✅ PDF berhasil dibaca. Ditemukan 15 halaman.


In [ ]:
# 2. Potong-potong teks menjadi "chunks" yang lebih kecil
# chunk_size=1000: setiap potongan maksimal 1000 karakter
# chunk_overlap=100: ada 100 karakter yang saling tumpang tindih antar potongan
#                    biar konteksnya nggak putus.
print("\n🔄 Memotong-motong teks menjadi chunks...")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)
print(f"✅ Teks berhasil dipotong menjadi {len(docs)} chunks.")

# Intip salah satu chunk untuk lihat hasilnya
print("\nContoh salah satu chunk:")
print(docs[5].page_content)


🔄 Memotong-motong teks menjadi chunks...
✅ Teks berhasil dipotong menjadi 49 chunks.

Contoh salah satu chunk:
sequence lengths, as memory constraints limit batching across examples. Recent work has achieved
significant improvements in computational efficiency through factorization tricks [21] and conditional
computation [32], while also improving model performance in case of the latter. The fundamental
constraint of sequential computation, however, remains.
Attention mechanisms have become an integral part of compelling sequence modeling and transduc-
tion models in various tasks, allowing modeling of dependencies without regard to their distance in
the input or output sequences [2, 19]. In all but a few cases [27], however, such attention mechanisms
are used in conjunction with a recurrent network.
In this work we propose the Transformer, a model architecture eschewing recurrence and instead
relying entirely on an attention mechanism to draw global dependencies between input and out

In [ ]:
# --- PART 1: Siapkan "Mesin Pembuat KTP" (Embedding Model) ---
# Kita gunakan model yang sama dengan proyek chatbot kita
print("🔄 Menyiapkan model embedding dari Hugging Face...")
embeddings = HuggingFaceEmbeddings(model_name='paraphrase-multilingual-MiniLM-L12-v2')
print("✅ Model embedding siap.")

🔄 Menyiapkan model embedding dari Hugging Face...


/tmp/ipython-input-9-240601179.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='paraphrase-multilingual-MiniLM-L12-v2')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Model embedding siap.


In [ ]:
# --- PART 2: Buat & Simpan "Perpustakaan" Vector Database ---
# Ini adalah langkah di mana kita mengubah semua 'docs' kita menjadi embeddings
# dan menyimpannya ke dalam FAISS. Ini mungkin butuh beberapa saat.
print("\n🔄 Membuat Vector Database dari text chunks... (Mungkin butuh beberapa menit)")
vectorstore = FAISS.from_documents(docs, embeddings)
print("✅ Vector Database berhasil dibuat!")


🔄 Membuat Vector Database dari text chunks... (Mungkin butuh beberapa menit)
✅ Vector Database berhasil dibuat!


In [ ]:
# (Opsional) Simpan "perpustakaan" ini ke laptop/Colab biar nggak perlu bikin ulang
vectorstore.save_local("faiss_index_research_assistant")
print("\n✅ Index Vector Database berhasil disimpan secara lokal.")


✅ Index Vector Database berhasil disimpan secara lokal.


In [ ]:
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from google.colab import userdata

In [ ]:
# --- PART 1: Siapkan "Kunci" untuk Panggil LLM ---
# Kita tetap pakai Groq yang super cepat
try:
    groq_api_key = userdata.get('GROQ_API_KEY')
    if not groq_api_key:
        st.warning("GROQ_API_KEY tidak ditemukan!")
except Exception as e:
    print(f"Pastikan GROQ_API_KEY sudah ada di Colab Secrets!")

In [ ]:
# --- PART 2: Siapkan "Otak" LLM-nya ---
# Kita inisialisasi model Llama3 dari Groq
print("🔄 Menyiapkan LLM dari Groq...")
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name='qwen-qwq-32b',
    temperature=0.7
)
print("✅ LLM siap.")

🔄 Menyiapkan LLM dari Groq...
✅ LLM siap.


In [ ]:
# --- PART 3: Buat "Template Perintah" ---
# Ini adalah instruksi yang akan kita berikan ke LLM
prompt_template = """
Gunakan potongan konteks berikut untuk menjawab pertanyaan.
Jika kamu tidak tahu jawabannya, katakan saja kamu tidak tahu, jangan mencoba mengarang jawaban.
Jawab dengan ringkas dan jelas dalam Bahasa Indonesia.

Konteks: {context}

Pertanyaan: {question}

Jawaban:
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
# --- PART 4: Rakit "Rantai Logika" RAG ---
print("\n🔄 Merakit Rantai Logika RAG...")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # "stuff" adalah metode paling simpel, dia "menjejalkan" semua konteks ke prompt
    retriever=vectorstore.as_retriever(), # Gunakan perpustakaan FAISS kita sebagai pencari
    return_source_documents=True, # Kita minta dia kasih tau "contekan" mana yang dia pake
    chain_type_kwargs={"prompt": PROMPT}
)
print("✅ Rantai Logika RAG siap!")


🔄 Merakit Rantai Logika RAG...
✅ Rantai Logika RAG siap!


In [ ]:
# =======================================================
#  RUANG OBROLAN INTERAKTIF DENGAN AI LIBRARIAN
# =======================================================

# Pastikan semua variabel sebelumnya sudah ada
# (llm, qa_chain, dll.)

print("✅ 'AI Librarian' sudah online!")
print("   Ketik pertanyaanmu, atau ketik 'exit' atau 'quit' untuk selesai.")
print("-" * 60)

# Loop akan terus berjalan sampai dihentikan
while True:
    # 1. Terima input dari lo sebagai user
    user_question = input("Kamu: ")

    # 2. Cek apakah lo mau udahan
    if user_question.lower() in ["exit", "quit"]:
        print("\nAI Librarian: Oke, bro. Senang bisa bantu. Sampai jumpa lagi!")
        break

    # 3. Panggil RAG chain kita untuk memproses pertanyaan
    #    (Ini akan memakan waktu beberapa detik)
    result = qa_chain({"query": user_question})

    # 4. Tampilkan jawaban dari AI
    print("\nAI Librarian:")
    print(result["result"])
    print("-" * 60)

✅ 'AI Librarian' sudah online!
   Ketik pertanyaanmu, atau ketik 'exit' atau 'quit' untuk selesai.
------------------------------------------------------------
Kamu: apa topik utama yang ada dalam paper ini?

AI Librarian:
<think>
Pertama, saya membaca pertanyaan yang ditanyakan, yaitu tentang topik utama dalam paper yang diberikan. Saya melihat konteks yang diberikan, yang mencakup abstrak, referensi, dan beberapa bagian lain dari paper tersebut.

Di dalam abstrak, paper ini membahas tentang model transduksi urutan yang dominan yang biasanya menggunakan jaringan saraf berulir atau konvolusi dengan encoder dan decoder. Namun, paper ini memperkenalkan arsitektur jaringan baru bernama Transformer yang hanya menggunakan mekanisme perhatian (attention) tanpa menggunakan rekurensi atau konvolusi.

Selanjutnya, saya melihat bahwa paper ini juga membahas tentang visualisasi perhatian (attention visualizations), yang menunjukkan bagaimana mekanisme perhatian bekerja dalam model, terutama pada 